# 2nd Notebook for multiple linear regression models

In [28]:
# Standard Packages
import pandas as pd
import numpy as np

# Viz Packages
import seaborn as sns
import matplotlib.pyplot as plt

# Scipy Stats
import scipy.stats as stats 

# Statsmodel Api
import statsmodels.api as sm
from statsmodels.formula.api import ols

# SKLearn Modules
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.feature_selection import f_regression

# Notebook Options
import warnings
warnings.filterwarnings("ignore", category= FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pd.options.display.max_columns = None
pd.options.display.width = None

In [29]:
mlr_baseline_df = pd.read_csv('../data/baseline_500k_to_15mil.csv')
mlr_baseline_df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,nuisance,view,condition,grade,heat_source,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,lat,long,renovated,zip,month,year
0,675000.0,4,1.0,1180,7140,1.0,0,0,0,0,4,7,Gas,PUBLIC,1180,0,0,40,1969,47.461975,-122.190520,0,98055,5,2022
1,920000.0,5,2.5,2770,6703,1.0,0,0,1,2,3,7,Oil,PUBLIC,1570,1570,0,240,1950,47.711525,-122.355910,0,98133,12,2021
2,775000.0,3,3.0,2160,1400,2.0,0,0,0,2,3,9,Gas,PUBLIC,1090,1070,200,270,2010,47.566110,-122.290200,0,98118,12,2021
3,592500.0,2,2.0,1120,758,2.0,0,0,1,0,3,7,Electricity,PUBLIC,1120,550,550,30,2012,47.532470,-122.071880,0,98027,8,2021
4,625000.0,2,1.0,1190,5688,1.0,0,0,1,0,3,7,Electricity,PUBLIC,1190,0,300,0,1948,47.763470,-122.340155,0,98133,7,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26116,719000.0,3,2.5,1270,1141,2.0,0,0,0,0,3,8,Gas,PUBLIC,1050,420,200,60,2007,47.690440,-122.370620,0,98117,10,2021
26117,1555000.0,5,2.0,1910,4000,1.5,0,0,0,0,4,8,Oil,PUBLIC,1600,1130,0,210,1921,47.664740,-122.329400,0,98103,11,2021
26118,1313000.0,3,2.0,2020,5800,2.0,0,0,0,1,3,7,Gas,PUBLIC,2020,0,0,520,2011,47.565610,-122.388510,0,98116,6,2021
26119,800000.0,3,2.0,1620,3600,1.0,0,0,1,0,3,7,Gas,PUBLIC,940,920,240,110,1995,47.610395,-122.295850,0,98122,5,2022


In [30]:
school_scores_df = pd.read_csv('../data/school_scores_by_zip.csv')
school_scores_df

,Unnamed: 0,ZIPCODE,percent_met_standard,district
0,0,98034,0.720261,LAKE WASHINGTON
1,1,98074,0.720261,LAKE WASHINGTON
2,2,98034,0.720261,LAKE WASHINGTON
3,3,98053,0.720261,LAKE WASHINGTON
4,4,98074,0.720261,LAKE WASHINGTON
...,...,...,...,...
649,649,98070,0.589803,VASHON ISLAND
650,650,98070,0.589803,VASHON ISLAND
651,651,98070,0.589803,VASHON ISLAND
652,652,98070,0.589803,VASHON ISLAND


In [31]:
school_scores_df = school_scores_df[['ZIPCODE', 'percent_met_standard']]
school_scores_df

,ZIPCODE,percent_met_standard
0,98034,0.720261
1,98074,0.720261
2,98034,0.720261
3,98053,0.720261
4,98074,0.720261
...,...,...
649,98070,0.589803
650,98070,0.589803
651,98070,0.589803
652,98070,0.589803


In [32]:
school_scores_df['zip'] = school_scores_df['ZIPCODE']
school_scores_df = school_scores_df[['percent_met_standard','zip']]
school_scores_df

/var/folders/wf/s7j2jn9j1ns035j4rx9g3rzr0000gn/T/ipykernel_5774/2146252363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_scores_df['zip'] = school_scores_df['ZIPCODE']


,percent_met_standard,zip
0,0.720261,98034
1,0.720261,98074
2,0.720261,98034
3,0.720261,98053
4,0.720261,98074
...,...,...
649,0.589803,98070
650,0.589803,98070
651,0.589803,98070
652,0.589803,98070


In [33]:
merged_baseline = pd.merge(mlr_baseline_df,school_scores_df, on='zip', how='left')
merged_baseline

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,nuisance,view,condition,grade,heat_source,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,lat,long,renovated,zip,month,year,percent_met_standard
0,675000.0,4,1.0,1180,7140,1.0,0,0,0,0,4,7,Gas,PUBLIC,1180,0,0,40,1969,47.461975,-122.19052,0,98055,5,2022,0.368621
1,675000.0,4,1.0,1180,7140,1.0,0,0,0,0,4,7,Gas,PUBLIC,1180,0,0,40,1969,47.461975,-122.19052,0,98055,5,2022,0.368621
2,675000.0,4,1.0,1180,7140,1.0,0,0,0,0,4,7,Gas,PUBLIC,1180,0,0,40,1969,47.461975,-122.19052,0,98055,5,2022,0.368621
3,920000.0,5,2.5,2770,6703,1.0,0,0,1,2,3,7,Oil,PUBLIC,1570,1570,0,240,1950,47.711525,-122.35591,0,98133,12,2021,0.555580
4,920000.0,5,2.5,2770,6703,1.0,0,0,1,2,3,7,Oil,PUBLIC,1570,1570,0,240,1950,47.711525,-122.35591,0,98133,12,2021,0.555580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263371,800000.0,3,2.0,1620,3600,1.0,0,0,1,0,3,7,Gas,PUBLIC,940,920,240,110,1995,47.610395,-122.29585,0,98122,5,2022,0.555580
263372,800000.0,3,2.0,1620,3600,1.0,0,0,1,0,3,7,Gas,PUBLIC,940,920,240,110,1995,47.610395,-122.29585,0,98122,5,2022,0.555580
263373,775000.0,3,2.5,2570,2889,2.0,0,0,0,0,3,8,Gas,PUBLIC,1830,740,480,100,2006,47.449490,-122.18908,0,98055,2,2022,0.368621
263374,775000.0,3,2.5,2570,2889,2.0,0,0,0,0,3,8,Gas,PUBLIC,1830,740,480,100,2006,47.449490,-122.18908,0,98055,2,2022,0.368621


In [34]:
mlr_baseline_df

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,greenbelt,nuisance,view,condition,grade,heat_source,sewer_system,sqft_above,sqft_basement,sqft_garage,sqft_patio,yr_built,lat,long,renovated,zip,month,year
0,675000.0,4,1.0,1180,7140,1.0,0,0,0,0,4,7,Gas,PUBLIC,1180,0,0,40,1969,47.461975,-122.190520,0,98055,5,2022
1,920000.0,5,2.5,2770,6703,1.0,0,0,1,2,3,7,Oil,PUBLIC,1570,1570,0,240,1950,47.711525,-122.355910,0,98133,12,2021
2,775000.0,3,3.0,2160,1400,2.0,0,0,0,2,3,9,Gas,PUBLIC,1090,1070,200,270,2010,47.566110,-122.290200,0,98118,12,2021
3,592500.0,2,2.0,1120,758,2.0,0,0,1,0,3,7,Electricity,PUBLIC,1120,550,550,30,2012,47.532470,-122.071880,0,98027,8,2021
4,625000.0,2,1.0,1190,5688,1.0,0,0,1,0,3,7,Electricity,PUBLIC,1190,0,300,0,1948,47.763470,-122.340155,0,98133,7,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26116,719000.0,3,2.5,1270,1141,2.0,0,0,0,0,3,8,Gas,PUBLIC,1050,420,200,60,2007,47.690440,-122.370620,0,98117,10,2021
26117,1555000.0,5,2.0,1910,4000,1.5,0,0,0,0,4,8,Oil,PUBLIC,1600,1130,0,210,1921,47.664740,-122.329400,0,98103,11,2021
26118,1313000.0,3,2.0,2020,5800,2.0,0,0,0,1,3,7,Gas,PUBLIC,2020,0,0,520,2011,47.565610,-122.388510,0,98116,6,2021
26119,800000.0,3,2.0,1620,3600,1.0,0,0,1,0,3,7,Gas,PUBLIC,940,920,240,110,1995,47.610395,-122.295850,0,98122,5,2022
